In [1]:
%matplotlib inline
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from fbprophet import Prophet
import pandas as pd
import numpy as np
import pyodbc
from sklearn.metrics import mean_absolute_error
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import pandas.io.sql as psql

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
cnxn = pyodbc.connect(xxxxxx)
cursor = cnxn.cursor()

In [3]:
sql = '''SELECT * FROM dbo.Dim_Borough'''
cursor.execute(sql)

In [4]:
data = cursor.fetchall()
# to fetch the results of a query, but not necessarily put it into a dataframe, use cursor.fetch...
for row in data:
    print row

(-1, -1, u'UNKNOWN')
(1, 1, u'MANHATTAN')
(2, 2, u'BRONX')
(3, 3, u'BROOKLYN')
(4, 4, u'QUEENS')
(5, 5, u'STATEN ISLAND')
(6, 9, u'OUT OF CITY')


In [6]:
sql_req_3 = '''
SELECT AVG(ConsumptionCF) AS y, ConsumptionDate as ds, Meter_No FROM dbo.Fact_Consumption
WHERE (ConsumptionDate BETWEEN '2013-03-01 00:00:00' AND '2016-01-01 00:00:00'
AND ConsumptionCF <> 0
AND Building_Class_ID = '{0}') GROUP BY Meter_No, ConsumptionDate'''.format('12')
cons3 = pd.read_sql(sql_req_3, cnxn)
#request training data - average daily consumption - for building type "12"

In [7]:
sql_req_4 = '''
SELECT AVG(ConsumptionCF) AS y, ConsumptionDate as ds, Meter_No FROM dbo.Fact_Consumption
WHERE (ConsumptionDate > '2016-01-01 00:00:00'
AND ConsumptionCF <> 0
AND Building_Class_ID = '{0}') GROUP BY Meter_No, ConsumptionDate'''.format('12')
cons4 = pd.read_sql(sql_req_4, cnxn)
#request test data - average daily consumption - for building type "12"

In [8]:
cons3["ds"] = pd.to_datetime(cons3["ds"])
cons3["ds"] = (cons3["ds"]).dt.date
cons4["ds"] = pd.to_datetime(cons4["ds"])
cons4["ds"] = (cons4["ds"]).dt.date

In [9]:
cons3a = cons3
cons4a = cons4

In [10]:
cons3a_92 = cons3a.y.quantile(.92)
cons3a_8 = cons3a.y.quantile(.08)
cons3a = cons3a.loc[cons3a['y'] < cons3a_92]
cons3a = cons3a.loc[cons3a['y'] > cons3a_8]
#trimming top and bottom .08 quartile to omit erratic values

In [11]:
cons4a_95 = cons4a.y.quantile(.95)
cons4a_5 = cons4a.y.quantile(.05)
cons4a = cons4a.loc[cons4a['y'] < cons4a_95]
cons4a = cons4a.loc[cons4a['y'] > cons4a_5]
#trimming top and bottom .05 quartile to omit erratic values

In [12]:
N_train = cons3a['Meter_No'].drop_duplicates().count()
#count the number of meters in the training set

In [13]:
N_test = cons4a['Meter_No'].drop_duplicates().count()
#count the number of meters in the test set

In [14]:
fb_cons3 = pd.DataFrame(cons3a.groupby(['ds'])['y'].mean())
fb_cons3['ds'] = fb_cons3.index
fb_cons3 = fb_cons3.reset_index(drop=True)
fb_cons3=fb_cons3.sort_values('ds')
fb_cons3["ds"] = pd.to_datetime(fb_cons3["ds"])
#formatting

In [15]:
fb_cons4 = pd.DataFrame(cons4a.groupby(['ds'])['y'].mean())
fb_cons4['ds'] = fb_cons4.index
fb_cons4 = fb_cons4.reset_index(drop=True)
fb_cons4=fb_cons4.sort_values('ds')
fb_cons4["ds"] = pd.to_datetime(fb_cons4["ds"])
#formatting

In [16]:
m = Prophet(yearly_seasonality=True)
m.fit(fb_cons3)
future = m.make_future_dataframe(periods=(fb_cons4.shape[0]))

In [17]:
forecast = m.predict(future)

In [19]:
predict_2016 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
cons_2016 = pd.concat([fb_cons3,fb_cons4],axis=0)
cons_2016["ds"] = pd.to_datetime(cons_2016["ds"])
all_2016 = pd.merge(cons_2016,predict_2016,left_on='ds',right_on='ds',how='left')
#concatenated table of date, predicted data, and confidence intervals for training and forecast data

In [20]:
t1=pd.DataFrame({'year': [2016],'month': [1],'day': [1]})
t1 = pd.to_datetime(t1).dt.date
#beginning of forecast data

In [21]:
y_16_true = all_2016.loc[all_2016['ds'] >= t1[0], ['y','ds']]
y_16_predict = all_2016.loc[all_2016['ds'] >= t1[0], ['yhat','ds','yhat_lower','yhat_upper']]
y_train_true = all_2016.loc[all_2016['ds'] < t1[0], ['y','ds']]
y_train_predict = all_2016.loc[all_2016['ds'] < t1[0], ['yhat','ds','yhat_lower','yhat_upper']]
#extracting variables for calculating mean absolute error

In [22]:
mae_test = mean_absolute_error(y_16_true.y, y_16_predict.yhat)
yyhatmean = ((y_16_true['y']+y_16_predict['yhat']).mean())/2
test_mae = ("{:.2%}".format(mae_test/yyhatmean))
#mean absolute error for forecast data

In [23]:
mae_train = mean_absolute_error(y_train_true.y, y_train_predict.yhat)
yyhatmean_train = ((y_train_true['y']+y_train_predict['yhat']).mean())/2
train_mae = ("{:.2%}".format(mae_train/yyhatmean_train))
#mean absolute error for training data

In [24]:
all_2016 = all_2016.sort_values('ds')

In [25]:
yhl_train = y_train_predict.yhat_lower.tolist()
yhu_train = y_train_predict.yhat_upper.tolist()
dates_train = y_train_predict.ds.tolist()
yhl_predict = y_16_predict.yhat_lower.tolist()
yhu_predict = y_16_predict.yhat_upper.tolist()
dates_predict = y_16_predict.ds.tolist()
#extracting variables for plottiing

In [ ]:
fig = plt.figure()

plt.plot(all_2016.ds,all_2016.y,'.k',label='Observed consumption')
plt.plot(y_train_predict.ds,y_train_predict.yhat,color = '#003399',label='Fit cons.')
plt.plot(y_16_predict.ds,y_16_predict.yhat,color = '#009900',label='Forecast cons.')
plt.fill_between(dates_train,yhl_train,yhu_train,color = '#66CCFF', alpha=0.7,label='Fit confidence interval')
plt.fill_between(dates_predict,yhl_predict,yhu_predict,color = '#28E528', alpha = 0.7,label='Forecast confidence interval')

plt.ylabel('Consumption (Cubic Feet)')
plt.ylim((min(y_16_predict.yhat_lower.min()-(3*np.std(y_16_predict.yhat_lower)),
     y_train_predict.yhat_lower.min()-(3*np.std(y_train_predict.yhat_lower))),
         max (y_16_predict.yhat_upper.max()+(3*np.std(y_16_predict.yhat_upper)),
     y_train_predict.yhat_upper.max()+(3*np.std(y_train_predict.yhat_upper)))))
#limits the y axis to show values within standard range for training and forecast data

plt.legend(loc='best')